In [1]:
import keras
import numpy as np
from keras.applications import vgg16
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.optimizers import Adam
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
            tf.config.experimental.set_virtual_device_configuration(
                        gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072)])
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    
      print(e)


Using TensorFlow backend.


1 Physical GPUs, 1 Logical GPUs


In [2]:
from collections import Counter
train_data = "D:/Dataset/Train"
validation_data = "D:/Dataset/Test"
batch_size = 13
img_height = 208
img_width = 176
classes=4
epoch=13
train_datagen= ImageDataGenerator(
        rescale= 1./255, 
        shear_range= 0.2,
        zoom_range= 0.2, 
        horizontal_flip= True)


validation_datagen= ImageDataGenerator(
        rescale= 1./255)
train_generator= train_datagen.flow_from_directory(
        train_data_dir, target_size= (img_height, img_width),
        batch_size= batch_size,
        class_mode='categorical')
validation_generator= validation_datagen.flow_from_directory(
        validation_data_dir, target_size= (img_height, img_width),
        batch_size= batch_size,
        class_mode= 'categorical')

counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}

base_model= keras.applications.vgg16.VGG16(weights= 'imagenet', include_top= False, input_shape=(img_height, img_width, 3))



                                                                                             




Found 5472 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [3]:
add_model= Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(250, activation= 'relu'))
add_model.add(Dense(4, activation='softmax'))
model= Model(inputs= base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-04, amsgrad=False),
              metrics=['accuracy'])

model.summary()
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 208, 176, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 208, 176, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 208, 176, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 104, 88, 64)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 104, 88, 128)      73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 104, 88, 128)      147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 52, 44, 128)       0   

In [ ]:
model.fit_generator(train_generator,
        epochs=epoch,
        validation_data=validation_generator,
        verbose=1,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        
        )

Epoch 1/17
194/195 [============================>.] - ETA: 0s - loss: 1.0071 - accuracy: 0.5630

In [7]:
#model.save("model1.h5")
from sklearn.metrics import classification_report, confusion_matrix
num_test=1279
Y_pred = model.predict_generator(validation_generator, num_test // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
#print('Classification Report')


Confusion Matrix
[[  5   0 126  48]
 [  0   0   9   3]
 [ 18   2 440 180]
 [ 12   1 315 120]]


In [8]:
target_names = ['MildDemented', 'Modearteddemented1', 'NonDemented','VeryMildDemented']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))


                    precision    recall  f1-score   support

      MildDemented       0.14      0.03      0.05       179
Modearteddemented1       0.00      0.00      0.00        12
       NonDemented       0.49      0.69      0.58       640
  VeryMildDemented       0.34      0.27      0.30       448

          accuracy                           0.44      1279
         macro avg       0.24      0.25      0.23      1279
      weighted avg       0.39      0.44      0.40      1279

